In [1]:
import pandas as pd
import numpy as np
import json
import re
import os

In [2]:
df = pd.read_csv('output_qwen32b.csv')

In [ ]:
def transform_row(row):
    row = row.map(lambda x: np.nan if isinstance(x, str) and "No data available" in x else x)
    row = row.fillna("")

    # Total Energy Consumption - kwh, mwh, gwh, gj + thousand, million -> mwh
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s]+)', row['Total Energy Consumption in Production'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s]+',row['Total Energy Consumption in Production'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if len(units) > 1:
            mult, unit = units[0].lower(), units[1].lower()
        else:
            unit = units[0].lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['kwh', 'mwh', 'gwh', 'gj']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
            if unit == 'kwh':
                num /= 1000.0
            if unit == 'gwh':
                num *= 1000.0
            if unit == 'gj':
                num /= 3.6
    else:
        num = np.nan
    row['Total Energy Consumption in Production'] = num

    # Energy Consumption per Vehicle Production - kwh/vehicle, mwh/vehicle, mj/vehicle -> mwh/vehicle
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s/]+)', row['Energy Consumption per Vehicle Production'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s/]+', row['Energy Consumption per Vehicle Production'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if len(units) > 1:
            mult, unit = units[0].lower(), units[1].lower()
        else:
            unit = units[0].lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['kwh', 'mwh', 'gwh', 'gj']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
            if unit == 'kwh/vehicle':
                num /= 1000.0
            if unit == 'mj/vehicle':
                num *= 3600
            if num < 1.5:
                num = np.nan
    else:
        num = np.nan
    row['Energy Consumption per Vehicle Production'] = num

    # Total Water Usage - m³, megaliters, gallons, kl, metric tons, ML + thousand, million -> m³
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Total Water Usage'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Total Water Usage'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if len(units) > 1:
            mult, unit = units[0].lower(), units[1].lower()
        else:
            unit = units[0].lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['m³', 'megaliters', 'gallons', 'kl', 'metric tons', 'ml']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
            if unit == 'megaliters' or unit == 'ml':
                num *= 1000
            if unit == 'gallons':
                num *= 0.00378541
    else:
        num = np.nan
    row['Total Water Usage'] = num

    # Total Wastewater Volume Generated - m³, gallons, liters -> m³
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Total Wastewater Volume Generated'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Total Wastewater Volume Generated'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if len(units) > 1:
            mult, unit = units[0].lower(), units[1].lower()
        else:
            unit = units[0].lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['m³', 'gallons', 'liters']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
            if unit == 'liters':
                num *= 0.001
            if unit == 'gallons':
                num *= 0.00378541
    else:
        num = np.nan
    row['Total Wastewater Volume Generated'] = num

    # Water Recycling and Reuse Rate % -> ratio
    tmp = re.sub(" ", "", re.sub("\\\\","",re.sub("[$]","",row['Water Recycling and Reuse Rate'])))
    if '%' in tmp:
        num = np.mean(np.array(re.findall(r"([\d.]+)%", tmp)).astype('float'))/100.0
    else:
        num = np.nan
    if num >= 1.5:
        num = np.nan
    row['Water Recycling and Reuse Rate'] = np.min([num, 1.00])

    # Total GHG Emissions - Metric Tons, T, Kton, KG -> Metric Tons
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Total GHG Emissions'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Total GHG Emissions'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['metric tons co', 'kg co', 'kton co', 'tonnes co']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
            if unit == 'kg co':
                num *= 1000
            if unit == 'kton co':
                num *= 0.001
    else:
        num = np.nan
    row['Total GHG Emissions'] = num

    # GHG Emissions and Intensity per Vehicle - T, Kg -> T
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['GHG Emissions and Intensity per Vehicle'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['GHG Emissions and Intensity per Vehicle'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['t co', 'kg co']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
            if unit == 'kg co':
                num *= 0.001
    else:
        num = np.nan
    row['GHG Emissions and Intensity per Vehicle'] = num

    # Total Manufacturing Waste Generation - Metric Tons, Tons, Kg -> Metric Tons
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Total Manufacturing Waste Generation'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Total Manufacturing Waste Generation'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['metric tons', 'kg', 'tons', 't']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
            if unit == 'kg':
                num *= 0.001
    else:
        num = np.nan
    row['Total Manufacturing Waste Generation'] = num

    # Waste Recycling and Diversion Rate % -> ratio
    tmp = re.sub(" ", "", re.sub("\\\\","",re.sub("[$]","",row['Waste Recycling and Diversion Rate'])))
    if '%' in tmp:
        num = np.mean(np.array(re.findall(r"([\d.]+)%", tmp)).astype('float'))/100.0
    else:
        num = np.nan
    if num >= 1.5:
        num = np.nan
    row['Waste Recycling and Diversion Rate'] = np.min([num,1.00]) 
    
    # Battery Recycling Rate % -> ratio
    tmp = re.sub(" ", "", re.sub("\\\\","",re.sub("[$]","",row['Battery Recycling Rate'])))
    if '%' in tmp:
        num = np.mean(np.array(re.findall(r"([\d.]+)%", tmp)).astype('float'))/100.0
    else:
        num = np.nan
    if num >= 1.5:
        num = np.nan
    row['Battery Recycling Rate'] = np.min([num,1.00]) 

    # Employee Count
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Employee Count'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Employee Count'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['count']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
    else:
        num = np.nan
    if num <= 0:
        num = np.nan
    row['Employee Count'] = [int(num) if not pd.isna(num) else num][0]

    # Employee Turnover Rate % -> ratio
    tmp = re.sub(" ","",re.sub("\\\\","",re.sub("[$]","",row['Employee Turnover Rate'])))
    if '%' in tmp:
        num = np.mean(np.array(re.findall(r"([\d.]+)%", tmp)).astype('float'))/100.0
    else:
        num = np.nan
    if num >= 1.5:
        num = np.nan
    row['Employee Turnover Rate'] = np.min([num,1.00]) 

    # Number of Workplace Accidents
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Number of Workplace Accidents'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Number of Workplace Accidents'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['count']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
    else:
        num = np.nan
    if num <= 0:
        num = np.nan
    row['Number of Workplace Accidents'] = [int(num) if not pd.isna(num) else num][0]

    # Employee Injury Rate
    tmp = re.sub(" ","",re.sub("\\\\","",re.sub("[$]","",row['Employee Injury Rate'])))
    if '%' in tmp:
        num = np.mean(np.array(re.findall(r"([\d.]+)%", tmp)).astype('float'))
    elif bool(re.fullmatch(r"\d+", tmp)):
        num = np.mean(np.array(re.findall(r"([\d.]+)", tmp)).astype('float'))
    else:
        num = np.nan
    row['Employee Injury Rate'] = num

    # Average Training Hours/Employee
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Average Training Hours/Employee'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Average Training Hours/Employee'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['hours']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
    else:
        num = np.nan
    if num <= 0:
        num = np.nan
    row['Average Training Hours/Employee'] = num

    # Training Investment/Employee
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³()]+)', row['Training Investment/Employee'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³()]+', row['Training Investment/Employee'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", "").replace("$","").replace("\\\\",""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['currency (usd)']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
    else:
        num = np.nan
    if num <= 0:
        num = np.nan
    row['Training Investment/Employee'] = num

    # Workforce Gender Ratios % -> ratio
    tmp = re.sub(" ","",re.sub("\\\\","",re.sub("[$]","",row['Workforce Gender Ratios'])))
    if '%' in tmp:
        num = np.mean(np.array(re.findall(r"([\d.]+)%", tmp)).astype('float'))/100.0
    elif bool(re.fullmatch(r"\d+", tmp)):
        num = np.mean(np.array(re.findall(r"([\d.]+)", tmp)).astype('float'))/100.0
    else:
        num = np.nan
    if num >= 1.5:
        num = np.nan
    row['Workforce Gender Ratios'] = np.min([num,1.00]) 

    # Workforce Minority Ratios % -> ratio
    tmp = re.sub(" ","",re.sub("\\\\","",re.sub("[$]","",row['Workforce Minority Ratios'])))
    if '%' in tmp:
        num = np.mean(np.array(re.findall(r"([\d.]+)%", tmp)).astype('float'))/100.0
    elif bool(re.fullmatch(r"\d+", tmp)):
        num = np.mean(np.array(re.findall(r"([\d.]+)", tmp)).astype('float'))/100.0
    else:
        num = np.nan
    if num >= 1.5:
        num = np.nan
    row['Workforce Minority Ratios'] = np.min([num,1.00]) 
    
    # Number of Corruption Incidents
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Number of Corruption Incidents'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Number of Corruption Incidents'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['count']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
    else:
        num = np.nan
    if num <= 0:
        num = np.nan
    row['Number of Corruption Incidents'] = [int(num) if not pd.isna(num) else num][0]
    
    # Anti-corruption Compliance Rate % -> ratio
    tmp = re.sub(" ","",re.sub("\\\\","",re.sub("[$]","",row['Anti-corruption Compliance Rate'])))
    if '%' in tmp:
        num = np.mean(np.array(re.findall(r"([\d.]+)%", tmp)).astype('float'))/100.0
    elif bool(re.fullmatch(r"\d+", tmp)):
        num = np.mean(np.array(re.findall(r"([\d.]+)", tmp)).astype('float'))/100.0
    else:
        num = np.nan
    if num >= 1.5:
        num = np.nan
    row['Anti-corruption Compliance Rate'] = np.min([num,1.00]) 

    # Number of Anti-competitive Practices
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Number of Anti-competitive Practices'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Number of Anti-competitive Practices'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['count']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
    else:
        num = np.nan
    if num <= 0:
        num = np.nan
    row['Number of Anti-competitive Practices'] = [int(num) if not pd.isna(num) else num][0]

    # Training Investment/Employee
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³()]+)', row['Monetary Value of Fines Imposed'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³()]+', row['Monetary Value of Fines Imposed'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", "").replace("$","").replace("\\\\",""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['currency (usd)']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
    else:
        num = np.nan
    if num <= 0:
        num = np.nan
    row['Monetary Value of Fines Imposed'] = num

    # Political Contributions and Lobbying Expenditures
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³()]+)', row['Political Contributions and Lobbying Expenditures'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³()]+', row['Political Contributions and Lobbying Expenditures'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", "").replace("$","").replace("\\\\",""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['currency (usd)']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
    else:
        num = np.nan
    if num <= 0:
        num = np.nan
    row['Political Contributions and Lobbying Expenditures'] = num

    # Number of Marketing Compliance and Ethical Advertising Violation Incidents
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Number of Marketing Compliance and Ethical Advertising Violation Incidents'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Number of Marketing Compliance and Ethical Advertising Violation Incidents'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['count']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
    else:
        num = np.nan
    if num <= 0:
        num = np.nan
    row['Number of Marketing Compliance and Ethical Advertising Violation Incidents'] = [int(num) if not pd.isna(num) else num][0]

    # Sales-weighted Average Fuel Economy/Emissions
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Sales-weighted Average Fuel Economy/Emissions'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Sales-weighted Average Fuel Economy/Emissions'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['mpg', 'gco']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
            if unit == 'gco':
                num = 8887/(num*1.609)
    else:
        num = np.nan
    row['Sales-weighted Average Fuel Economy/Emissions'] = num

    # Zero Emission and Alternative Fuel Vehicle Sales
    units = re.findall(r'\d[\d,\s.]+([A-Za-z\s³]+)', row['Zero Emission and Alternative Fuel Vehicle Sales'])
    num = re.findall(r'(\d[\d,\s.]+)[A-Za-z\s³]+', row['Zero Emission and Alternative Fuel Vehicle Sales'])
    if units and num:
        if len(units) > 1 and re.sub(" ","",units[0]) == "":
                units = units[1].split()
        else:
            units = units[0].split()
        tmp = 0
        for n in num:
            tmp += float(n.replace(",","").replace(" ", ""))
        num = tmp / len(num)
        mult, unit = '', ''
        if units[0] in ['thousand', 'million', 'billion']:
            mult = units[0].lower()
            unit = ' '.join(units[1:]).lower()
        else:
            unit = ' '.join(units).lower()
        if mult not in ['thousand', 'million', 'billion'] and unit not in ['vehicle units']:
            num = np.nan
        else:
            if mult == 'thousand':
                num *= 1000
            if mult == 'million':
                num *= 1000000
            if mult == 'billion':
                num *= 1000000000
    else:
        num = np.nan
    if num <= 0:
        num = np.nan
    row['Zero Emission and Alternative Fuel Vehicle Sales'] = [int(num) if not pd.isna(num) else num][0]

    return row

In [4]:
df_t = df.apply(transform_row, axis=1)

In [5]:
with open('linkedin_scrape.json', 'r') as file:
    linkedin_scrape = json.load(file)

In [6]:
df_t['size'] = linkedin_scrape['company_size']

In [7]:
def group_company(row):
    if row['size'] in ['10K+', '5K-10K']:
        row['size'] = 'Large'
    elif row['size'] in ['501-1K', '1K-5K']:
        row['size'] = 'Medium'
    else:
        row['size'] = 'Small'
    return row

In [8]:
df_t = df_t.apply(group_company, axis=1)

In [9]:
quantitatives = list(df_t.columns[1:11]) + list(df_t.columns[12:27])
qualitatives = list(df_t.columns[-10:-1])

In [10]:
df_st = pd.read_csv('standard.csv')

In [11]:
def normalize_qn1(row):
    signs = [-1,-1,-1,-1,1,-1,-1,-1,1,1,-1,-1,-1,1,1,0,0,-1,1,-1,-1,-1,-1,-1,1]
    cond = df_st[df_st['size'] == row['size']]
    for i,col in enumerate(quantitatives):
        if col not in ['Workforce Gender Ratios', 'Workforce Minority Ratios']:
            row[col] = float(row[col] - cond[col])*signs[i]
        else:
            row[col] = float(1 - (row[col] - cond[col])**2)
    return row
def normalize_qn2(df):
    for col in quantitatives:
        tmp = df[col]
        tmp_max = np.max(df[col])
        tmp_min = np.min(df[col])
        df[col] = (2*(tmp - tmp_min) - tmp_max + tmp_min)/(tmp_max - tmp_min)
    return df

def noramlize_quantitative(df):
    df_res = df.apply(normalize_qn1, axis=1)
    df_res = normalize_qn2(df_res)
    return df_res

def restructure_ql(df, sentiment):
    df['filename'] = df_t['filename']
    for company, senti in sentiment.items():
        row = df[df['filename'] == company]
        for indi, values in senti.items():
            tmp = values['sentiment'].lower()
            if tmp == 'positive':
                tmp = 1
            elif tmp == 'negative':
                tmp = -1
            else:
                tmp = 0
            row[indi] = tmp
        df[df['filename'] == company] = row
    df = df.drop(columns=['filename'], errors='ignore')
    return df

In [12]:
df_quanti = noramlize_quantitative(df_t)
df_quanti = df_quanti[['filename'] + quantitatives]

C:\Users\jaeju\AppData\Local\Temp\ipykernel_13308\603314450.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  row[col] = float(row[col] - cond[col])*signs[i]
C:\Users\jaeju\AppData\Local\Temp\ipykernel_13308\603314450.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  row[col] = float(1 - (row[col] - cond[col])**2)


In [13]:
with open('sentiment_analysis_results.json', 'r') as file:
    sentiment = json.load(file)

In [14]:
df_quali = pd.DataFrame(np.nan, index=range(61), columns=qualitatives)
df_quali = restructure_ql(df_quali, sentiment)

C:\Users\jaeju\AppData\Local\Temp\ipykernel_13308\603314450.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[indi] = tmp
C:\Users\jaeju\AppData\Local\Temp\ipykernel_13308\603314450.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[indi] = tmp
C:\Users\jaeju\AppData\Local\Temp\ipykernel_13308\603314450.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [15]:
df_norm = pd.concat([df_quanti, df_quali], axis=1).fillna(-0.2)

In [16]:
with open('esg_weights.json', 'r') as file:
    esg_weights = json.load(file)

In [17]:
def fit_minmax(row):
    for score in ['esg_score', 'e_score', 's_score', 'g_score']:
        if row[score] > 100:
            row[score] = 100
        if row[score] < 0:
            row[score] = 0
        row[score] = np.round(row[score], 2)
    return row

In [18]:
df_norm['e_score'] = df_norm.iloc[:,[1,2,3,4,5,6,7,8,9,10,24,25,26,27,28,33,34]] @ esg_weights['E']['Coef'] + esg_weights['E']['Intercept']
df_norm['s_score'] = df_norm.iloc[:,[11,12,13,14,15,16,17,29,30]] @ esg_weights['S']['Coef'] + esg_weights['S']['Intercept']
df_norm['g_score'] = df_norm.iloc[:,[18,19,20,21,22,23,31,32]] @ esg_weights['G']['Coef'] + esg_weights['G']['Intercept']
df_norm['esg_score'] = df_norm[['e_score', 's_score', 'g_score']] @ esg_weights['ESG']['Coef'] + esg_weights['ESG']['Intercept']
df_scored = df_norm.apply(fit_minmax, axis=1)

In [19]:
def parse_filename(filename):
    try:
        # Remove path and extension
        clean_name = os.path.splitext(os.path.basename(filename))[0]
        
        # Extract year using regex
        year_match = re.search(r'-(\d{4})(\s|$)', clean_name)
        year = int(year_match.group(1)) if year_match else None
        
        # Remove year and split remaining parts
        base = re.sub(r'-\d{4}.*', '', clean_name)
        parts = base.split('-')
        
        # Handle different industry-company patterns
        if len(parts) >= 2:
            if 'Major' in parts[1]:
                industry = '-'.join(parts[0:2]).strip()
                company = '-'.join(parts[2:]).strip()
            else:
                industry = parts[0].strip()
                company = '-'.join(parts[1:]).strip()
        else:
            industry = base.strip()
            company = 'Not Reported'
        
        return pd.Series([industry, company, year])
    
    except Exception as e:
        print(f"Error parsing {filename}: {str(e)}")
        return pd.Series([None, None, None])

In [20]:
df_scored[['Industry', 'Company', 'Year']] = df_scored['filename'].apply(parse_filename)

In [24]:
df_t[['Industry', 'Company', 'Year']] = df_t['filename'].apply(parse_filename)

In [22]:
df_scored.to_csv('final_output.csv')

In [25]:
df_t.to_csv('nonnormalized.csv')